In [1]:
%xmode plain
import glob
import cv2
import numpy as np

Exception reporting mode: Plain


In [ ]:
python --version

In [3]:
suits_dict = {0:'Diamond',1:'Spade', 2:'Heart', 3:'Club'}

In [4]:
# mask = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06115-lbl.png',1)
# img = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06115.png',1)
# mask = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06115-lbl.png',1)
# img = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06115.png',1)
mask = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06891-lbl.png',1)
img = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/06891.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(mask,127,255,0)
_,img = cv2.threshold(img,127,255,0)
im2 ,num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cimg = cv2.drawContours(img,num_cards,-1,(0,255,0),1)
# cv2.imshow('teg', cimg)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
def preprocess_card(contour, img):
    
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.09*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(contour)
    average = np.sum(pts, axis=0)/len(pts)
    img = cv2.blur(img, (3,3))
    warp = flattener(img, pts, w, h)
    # blur blur blur
#     cv2.imshow('ee',warp)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    warp = cv2.blur(warp, (5,5))
    ret,thresh = cv2.threshold(warp,127,255,cv2.THRESH_BINARY)
    im2 ,contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = np.array([cv2.contourArea(contours[i], True) for i in range(len(contours))]) 
    
    print(np.percentile(area[1:],50))
    suit_contours = (area>np.percentile(area[1:],50)).nonzero()[0]
    suit_areas = area[suit_contours]
    z = [x for _,x in sorted(zip(area[suit_contours],suit_contours))]
    largest_suit_contour = z[-3]
    x,y,w,h = cv2.boundingRect(contours[largest_suit_contour])
    ret,th1 = cv2.threshold(warp,127,255,cv2.THRESH_BINARY)
    warp_c = cv2.rectangle(th1,(x,y),(x+w,y+h),(0,255,0),2)
    warp_c.shape
    cropped_suit = warp_c[y:y+h, x:x+w]
    
    return cropped_suit, len(suit_contours)-1

In [6]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]

    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]

    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

    return warp

In [ ]:
def find_rank(img_suit):
    suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
    for i,img in enumerate(suits):
        img = cv2.cvtColor(cv2.resize(img, (img_suit.shape[1], img_suit.shape[0])), cv2.COLOR_BGR2GRAY)
        suits[i] = cv2.blur(img,(5,5))
    
    minimum1 = 0
    idx1  = 0   
    ret = 0
    for i in range(len(suits)):
        ans = np.abs((img_suit + cv2.flip(suits[i], 0)))
        tans = ans[ans < 128].shape[0]
        print(i," ", tans)
        
        if tans > minimum1:
            minimum1 = tans
            idx1 = i
            ret = ans
    minimum2 = 0
    idx2 = 0


    for j in range(len(suits)):
        ans = np.abs(img_suit + suits[j])
        ans = ans[ans < 128].shape[0]
        print(j," - ", ans)
        if ans > minimum2:
            minimum2 = ans
            idx2 = j
    cv2.imshow('res0', img_suit)
    cv2.imshow('res1',2*img_suit + suits[0] + cv2.flip(suits[0], 0))
    cv2.imshow('res2',2*img_suit + suits[1] + cv2.flip(suits[1], 0))
    cv2.imshow('res3',2*img_suit + suits[2] + cv2.flip(suits[2], 0))
    cv2.imshow('res4',2*img_suit + suits[3] + cv2.flip(suits[3], 0))
    
#     cv2.imshow('rres1',2*img_suit + cv2.flip(suits[0], 0))
#     cv2.imshow('rres2',2*img_suit + cv2.flip(suits[1], 0))
#     cv2.imshow('rres3',2*img_suit + cv2.flip(suits[2], 0))
#     cv2.imshow('rres4',2*img_suit + cv2.flip(suits[3], 0))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    if minimum2 > minimum1:   
        return idx2
    else:    
        return idx1

In [ ]:
for i in range(len(num_cards)):
    cropped_suit, card_value = preprocess_card(num_cards[i], img)
    ans = find_rank(cropped_suit)
    print("Card Value: {} and suit {}".format(card_value,suits_dict[ans]))

792.0
0   788
1   870
2   874
3   823
0  -  788
1  -  893
2  -  907
3  -  847
